In [30]:
import os
import sys
from glob import glob

import numpy as np
import pandas as pd

import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

basic_packages = [os, sys, glob, np, pd, cv, Image, plt]
print("Packages Version Infos :")
for pname in basic_packages:
    try:
        print(f"\t{pname.__name__}\t:\t{pname.__version__}")
    except:
        print(f"\tmodule '{pname.__name__}' has no attribute '__version__'")

Packages Version Infos :
	module 'os' has no attribute '__version__'
	module 'sys' has no attribute '__version__'
	module 'glob' has no attribute '__version__'
	numpy	:	1.21.4
	pandas	:	1.3.4
	cv2	:	4.5.4
	PIL.Image	:	8.4.0
	module 'matplotlib.pyplot' has no attribute '__version__'


In [43]:
from os.path import split as spt
from os.path import join as jn

BASE = spt(spt(os.getcwd())[0])[0]
CONF = jn(BASE, "config")

/Volumes/WORK/Google/Colab-Server-VM/Waste-Recycling-Image-Segmentation


```bash
%load "{CONF}/annots-cv.py"
```

In [192]:
# %load "/Volumes/WORK/Google/Colab-Server-VM/Waste-Recycling-Image-Segmentation/config/annots-cv.py"
import numpy as np

import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

class annotCats:
    def __init__(self, image_path=None, label=None):
        self.colors = self.cvColor()
        self.label = label
        try:
            self.image = Image.open(image_path)
            self.image = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
        except:
            print("이미지 경로를 제대로 입력하세요.")
        
    def img2points(self, x, y, pcolor="RED", size=5, option=-1, check=True):
        if check:
            self.updated_img = self.image.copy()
        plt.grid(color=self.colors["WHITE"], linestyle="dashdot", linewidth=.5)
        cv.circle(self.updated_img, (x, y), size, self.colors[pcolor], option)
        plt.imshow(self.updated_img)
        if not check:
            return self.updated_img

    def img2rectangle(self, x1, y1, x2, y2, pcolor="RED", check=True, rect_lwidth=2):
        if check:
            self.updated_img = self.image.copy()
        plt.grid(color=self.colors["WHITE"], linestyle="dashdot", linewidth=.5)
        cv.rectangle(self.updated_img, (x1, y1), (x2, y2), self.colors[pcolor], linewidth=rect_lwidth)
        plt.imshow(self.updated_img)
        if not check:
            return self.updated_img 
        
    def cvColor(self):
        # R, G, B
        BLACK = (0, 0, 0)
        WHITE = (255, 255, 255)
        GRAY = (125, 125, 125)
        RED = (255, 0, 0)
        GREEN = (0, 255, 0)
        BLUE = (0, 0, 255)
        CYAN = (0, 255, 255)
        MAGENTA = (255, 0, 255)
        YELLOW = (255, 255, 0)
        PINK = (238, 130, 238)
        ORANGE = (255, 165, 0)
        MINT = (60, 179, 113)
        LAVENDER = (106, 90, 205)
        IVORY = (240, 240, 240)
        SALMON = (240, 150, 120)

        colors = {"RED":RED, "GREEN":GREEN, "BLUE":BLUE, \
                "MAGENTA":MAGENTA, "CYAN":CYAN, "YELLOW":YELLOW, \
                "WHITE":WHITE, "GRAY":GRAY, "BLACK":BLACK, \
                "PINK":PINK, "ORANGE":ORANGE, "MINT":MINT, \
                "LAVENDER":LAVENDER, "IVORY":IVORY, "SALMON":SALMON}
        return colors

In [193]:
TRAIN = jn(BASE, "assets/data/plastic-segmentation/Train")
TEST = jn(BASE, "assets/data/plastic-segmentation/Test")

PCLS = []
paths = {"image_path" : [], "annot_path": []}

for fpath in [TRAIN, TEST]:
    kind = os.path.basename(fpath)
    BDIR = sorted(os.listdir(fpath))
    for bdir in BDIR:
        try:
            pcls = str(bdir.lower())
            assert pcls.startswith("p")
            img_path = jn(fpath, bdir)
            img_paths = sorted(glob(img_path + "/*.jpg"))
            (paths['image_path']).extend(img_paths)
            PCLS.append(bdir)
        except:
            for pdir in np.unique(PCLS):
                annot_path = jn(fpath, bdir, pdir)
                annot_path = sorted(glob(annot_path + "/*.json"))
                (paths['annot_path']).extend(annot_path)

In [194]:
kind = pd.Series([fpath.split('/')[-3] for fpath in paths['image_path']], name="kind")
label = pd.Series([fpath.split('/')[-2] for fpath in paths['image_path']], name="label")
metainfo_id = pd.Series([(fpath.split('/')[-1]).split('_')[1] for fpath in paths['image_path']], name="metainfo_id").astype(int)
feats = pd.Series([(fpath.split('/')[-1]).split('_')[2][:-4] for fpath in paths['image_path']], name="feature").astype(int)

baseDF = pd.concat([kind, label, metainfo_id, feats, pd.DataFrame(paths)], axis=1)

In [195]:
print(len(baseDF['metainfo_id'].unique()))

pd.DataFrame(baseDF['metainfo_id'].value_counts().reset_index()).head(20)

44


,index,metainfo_id
0,58,295
1,76,294
2,56,277
3,54,270
4,50,258
5,34,183
6,35,160
7,74,152
8,88,152
9,37,150


In [196]:
pd.DataFrame(baseDF['metainfo_id'].value_counts().reset_index()).describe()

,index,metainfo_id
count,44.000000,44.000000
mean,35.159091,100.000000
std,26.909792,78.542834
min,2.000000,33.000000
25%,15.750000,42.500000
50%,26.500000,52.500000
75%,44.000000,142.000000
max,103.000000,295.000000


In [197]:
baseDF.columns

Index(['kind', 'label', 'metainfo_id', 'feature', 'image_path', 'annot_path'], dtype='object')

In [198]:
baseDF.describe()

,metainfo_id,feature
count,4400.000000,4400.000000
mean,48.299545,3147.959318
std,25.743039,2736.973045
min,2.000000,1.000000
25%,33.000000,207.000000
50%,46.000000,2769.000000
75%,62.000000,5506.000000
max,103.000000,10132.000000


In [199]:
pd.DataFrame(baseDF.groupby(["kind", "label", "metainfo_id"])["feature"].value_counts())

feature
kind  label metainfo_id feature         
Test  PE    74          1              1
                        3              1
                        5              1
                        7              1
                        9              1
...                                  ...
Train PS    42          7294           1
                        7482           1
                        7596           1
                        7605           1
                        7734           1

[4400 rows x 1 columns]

18시 이후부터 계속 진행하겠습니다.